In [1]:
%matplotlib widget
import pandas as pd
import numpy as np

from pathlib import Path

import matplotlib.pyplot as plt
import seaborn as sns

# local import
import lppydsmc as ld

In [2]:
dir_path = Path('../../../lppydsmc/simulations/results').resolve()

In [3]:
name_simulation = 'thruster_two_grids_background_gas' # 'thruster_two_grids_neutrals' # thruster_two_grids thruster_three_grids_neutrals thruster_three_grids

In [4]:
store = pd.HDFStore(dir_path/Path(name_simulation)/'monitoring.h5')
print(store.keys())

['/fluxes', '/out_particles', '/particles', '/wall_collisions']


# Parameters of the simulation

If you used the option `-s` (or `save`) when launching the simulation, it can be found under `<dir_path>/<name_simulation>/params.ini`.

In [5]:
time_step = 1e-07
steps_between_savings = 4
dsmc_grid_shape = np.array([16,5], dtype = int)
offsets = np.array([0,0])
system_size = np.array([0.016,0.005])
dx = dy = dz = 0.001
cell_volume = dx*dy*dz
particles_weight = 320000 # 32000000

# Analysis of the results

First, we will simply focus on getting the density and velocity functions out to launch on background gas a better simulation.

In [6]:
df_particles = store['particles']

In [7]:
evo_nb_particles = df_particles['species'].groupby(df_particles.index).count()
print(evo_nb_particles)

1         71
4        285
8        570
12       839
16       928
        ... 
984     1080
988     1072
992     1076
996     1083
1000    1071
Name: species, Length: 251, dtype: int64


In [8]:
fig, ax = plt.subplots(constrained_layout = True)
ax.plot(evo_nb_particles.index*time_step, evo_nb_particles)
ax.set_xlabel('time (s)')
ax.set_ylabel('Number of particles');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

The previous figure shows at what point we are arriving in a more steady state.

In [9]:
steady_state = int(2e-5/time_step)
# df_particles_steady = df_particles[df_particles.index > steady_state]

At this point, the goal is simply to create *density* and *dynamic* functions from the results in steady flow. 
To start with, we will select a mesh (in practice we use the same as for the DSMC), compute the values and interpolate them in a function that we save for later use.

In [10]:
interpolate = False
if(interpolate):
    X, Y, density_arr, dynamic_arr = ld.background_gas.interpolate.interpolate(df_particles, offsets, dsmc_grid_shape, system_size, cell_volume, particles_weight, steady_state = steady_state)
    np.save('gas_density_arr_{}.npy'.format(name_simulation), density_arr, allow_pickle = True)
    np.save('gas_dynamic_arr_{}.npy'.format(name_simulation), dynamic_arr, allow_pickle = True)
    np.save('x_arr_{}.npy'.format(name_simulation), X, allow_pickle = True)
    np.save('y_arr_{}.npy'.format(name_simulation), Y, allow_pickle = True)
    # ld.background_gas.interpolate.read_interpolation()

## Check output thruster

In [17]:
df_out = store['out_particles'] # df particles should be updated at every frame ... but anyway

In [13]:
df_out.head()

,x,y,vx,vy,vz,species
12,-0.000371,0.003926,-6698.941273,-819.996187,797.790037,0.0
12,-0.000286,0.000937,-6569.134074,735.507669,-3.435215,0.0
12,-0.000147,0.001488,-6609.413279,1406.166849,-205.416170,0.0
12,-0.000096,0.004890,-6677.381557,-449.168556,-138.178526,0.0
12,-0.000103,0.003390,-6661.864690,-1594.947177,436.379528,0.0


In [14]:
df_out.groupby('species').count()

,x,y,vx,vy,vz
species,,,,,
0.0,15612,15612,15612,15612,15612
1.0,1871,1871,1871,1871,1871


In [20]:
df_out.groupby('species').mean()

,x,y,vx,vy,vz
species,,,,,
0.0,0.011746,0.002489,11405.530871,-41.798310,-4.769201
1.0,0.016674,0.002488,21061.246738,-23.876742,24.776384


## Particles collisions with walls

In [21]:
df_wall_collisions = store['wall_collisions']

In [27]:
fig, ax = plt.subplots(constrained_layout = True)
sns.scatterplot(ax=ax, data = df_wall_collisions, x = 'x', y = 'y', hue = 'species', s = 0.3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='x', ylabel='y'>